## 4-1. 들어가며

렉처 노드에서는 공간 데이터의 개념, API 활용, 데이터 전처리, 다양한 라이브러리를 이용해 시각화하는 방법을 학습했습니다.

이번에는 배운 것을 응용해서 실제로 쏘카에서 진행하는 업무와 같이 아래 목록의 공간 데이터를 시각화해 보는 실습을 하도록 하겠습니다. 목록 아래에 시각화 결과 예시를 첨부하였습니다.

1.  서울시 성동구 공시지가 데이터 수집 및 시각화

![content img](https://d3s0tskafalll9.cloudfront.net/media/images/seoul_house_price_sujeong.max-800x600.jpg)

2.  서울 소재 대학교 위치 데이터 수집 및 시각화

![content img](https://d3s0tskafalll9.cloudfront.net/media/images/seoul_univ.max-800x600.png)

3.  전국주차장 데이터 수집 및 시각화

![content img](https://d3s0tskafalll9.cloudfront.net/media/images/parking_lot.max-800x600.png)

4.  서울시 구별 인구수 데이터 수집 및 시각화

![content img](https://d3s0tskafalll9.cloudfront.net/media/images/seoul_population_sujeong.max-800x600.jpg)

### 학습목표

___

-   다양한 공간 데이터를 수집하고 시각화한다.
-   쏘카존 데이터를 시각화한다.
-   쏘카존 데이터를 시각화한 자료에 추가한다.

### 목차

___

1.  서울시 성동구 공시지가 데이터 수집 및 시각화
2.  서울시 대학교 데이터 수집 및 시각화
3.  전국주차장 데이터 시각화
4.  서울시 구별 인구수 데이터 수집 및 시각화
5.  프로젝트

### 준비물

___

클라우드에 있는 파일을 사용하기 위해 `cloud shell`에서 아래 명령어를 입력해주시기 바랍니다.

```
$ mkdir -p ~/aiffel/socar_gis/data
$ ln -s ~/data/*  ~/aiffel/socar_gis/data/
```

## 4-2. 서울시 성동구 공시지가 데이터 수집 및 시각화

서울시 공시지가 API를 활용해서 성동구 공시지가를 시각화하겠습니다.

### 라이브러리 불러오기

___

먼저 필요한 라이브러리를 불러옵니다.

In [ ]:
### SGIS ACCESS TOKEN을 리턴 해주는 함수
def get_access_token(consumer_key, consumer_secret) -> None:
    temp_res = requests.get(
        url="https://sgisapi.kostat.go.kr/OpenAPI3/auth/authentication.json",
        params={
            "consumer_key": consumer_key,
            "consumer_secret": consumer_secret
        }
    )

    access_token = temp_res.json()["result"]["accessToken"]
  
    return access_token
print('슝~')

### 데이터 수집 및 전처리

___

Lecture 노드에서 SGIS API를 사용하기 전에 access token을 반환해주는 함수를 먼저 정의하고 진행했습니다. 기억을 상기시키기 위해 다시 한 번 정의해 보겠습니다.

In [ ]:
### SGIS ACCESS TOKEN을 리턴 해주는 함수
def get_access_token(consumer_key, consumer_secret) -> None:
    temp_res = requests.get(
        url="https://sgisapi.kostat.go.kr/OpenAPI3/auth/authentication.json",
        params={
            "consumer_key": consumer_key,
            "consumer_secret": consumer_secret
        }
    )

    access_token = temp_res.json()["result"]["accessToken"]
  
    return access_token
print('슝~')

다음에는 시에 속한 구이름을 리스트로 반환하는 함수 `get_si_info_list`를 정의하겠습니다.

In [ ]:
# 시에 속한 구이름을 리스트로 리턴 받는 함수
def get_si_info_list(consumer_key, consumer_secret, cd) -> None:
    ACCESS_TOKEN=get_access_token(consumer_key, consumer_secret)
    PARAMS={
        "accessToken": ACCESS_TOKEN,
        "cd": cd # 선택) 최대 5자리, non(default): 시도정보, 2자리: 시군구, 5자리: 읍면동
        # "pg_yn": , # 선택) 경계 미포함(default): 0, 경계 포함: 1
    }

    temp_res = requests.get(
        url="https://sgisapi.kostat.go.kr/OpenAPI3/addr/stage.json",
        params=PARAMS
    )

    temp_sido_info_list = temp_res.json()["result"]
    
    return temp_sido_info_list
print('슝~')

이제 서울시에 속하는 구들의 이름을 리스트로 가져오겠습니다.  
서울시에 어떤 구들이 있는지 확인만 해보고 어떤 구를 선택할 지 결정해봅시다.  
`consumer_key`, `consumer_secret` 변수에는 [SGIS API](https://sgis.kostat.go.kr/developer/html/home.html)에서 발급받은 본인 인증키 값을 입력하면 됩니다.

In [ ]:
# SGIS API 사이트의 본인의 consumer_key, consumer_secret를 입력합니다.
consumer_key = '13c365600dd54f4f99a7'       # 서비스ID
consumer_secret =  '4982fcdb06094e47b61b' # 보안Key

temp_seoul_info_list = get_si_info_list(consumer_key, consumer_secret, 11)
seoul_gu_name_list = [gu_name["addr_name"] for gu_name in temp_seoul_info_list ]
seoul_gu_name_list

서울시의 많은 구 중에서 저희는 **성동구**를 선택하겠습니다.  
다음은 서울 열린데이터 API를 활용해 서울시 성동구의 공시지가(원/㎡)를 리스트 형태로 받아온 후 DataFrame으로 생성하겠습니다.

아래 링크로 이동해 회원가입 및 인증키 신청을 진행하시면 됩니다. (서울시 열린데이터 광장 접속 시 chrome이 안된다면 익스플로러나 Edge를 사용하여 접속해 보세요. )

-   [서울시 공시지가 정보](http://data.seoul.go.kr/dataList/OA-1180/F/1/datasetView.do)

![content img](https://d3s0tskafalll9.cloudfront.net/media/images/seoulgonggaedeiteo1_injeungkisinceong.max-800x600.jpg)

회원가입 후 인증키 신청을 하면 아래와 같은 화면이 나옵니다.  
각 항목에 맞게 입력해주시면 인증키를 얻을 수 있습니다.  
사용 URL은 임의의 URL을 입력해주시면 됩니다 ex)[https://lms.aiffel.io/](https://lms.aiffel.io/)

![content img](https://d3s0tskafalll9.cloudfront.net/media/images/seoulgonggaedeiteo2_injeungkisinceong.max-800x600.jpg)

![content img](https://d3s0tskafalll9.cloudfront.net/media/images/seoulgonggaedeiteo3_injeungkihwagin.max-800x600.jpg)

**아래 코드 중 `KEY`라는 변수에 서울 열린데이터 광장의 본인의 인증키를 입력해주시기 바랍니다.**

In [ ]:
# 반복문 초기값
START_INDEX = 1
END_INDEX = 1000
request_code = "INFO-000"
temp_result_list = []

while True:
    if(request_code!="INFO-000"):
        break
    elif(request_code=="INFO-000"):
        # API 조회간격 조절
        time.sleep(0.2) # 0.2초 대기후 요청

    # 요청 파라미터
    """
    변수 KEY에는 서울 열린데이터 광장의 본인의 인증키를 적어주세요.
    """
    KEY = "5065425754706b6233305571467a54" # 서울 열린데이터 광장의 본인의 인증키
    TYPE = "json"
    SERVICE = "IndividuallyPostedLandPriceService"
    START_INDEX = START_INDEX + 1000
    END_INDEX = END_INDEX + 1000
    SIGUNGU_NM = "성동구"
    YEAR = 2020

    url = f"http://openapi.seoul.go.kr:8088/{KEY}/{TYPE}/{SERVICE}/{START_INDEX}/{END_INDEX}/{SIGUNGU_NM}/ / / / /{YEAR}"

    try:
        temp_res = requests.get(
            url=url
        )

        request_code = temp_res.json()["IndividuallyPostedLandPriceService"]["RESULT"]["CODE"]
      
        temp_res_list = temp_res.json()["IndividuallyPostedLandPriceService"]["row"]
        temp_result_list = temp_res_list + temp_result_list
    
    except:
        request_code = "INFO-200"


print(f"총 데이터수: {len(temp_result_list)}")

In [ ]:
sungdong_gu_giga_df = pd.DataFrame(temp_result_list)
sungdong_gu_giga_df.head()

공시지가의 연산을 위해 데이터 타입을 object에서 int로 변경한 후, 성동구 동별 공시지가(원/㎡)를 집계하여 평균지가 컬럼에 추가합니다.

In [ ]:
sungdong_gu_giga_df["JIGA"] = sungdong_gu_giga_df["JIGA"].astype(int)
sungdong_gu_giga_df.info()

In [ ]:
sungdong_gu_dong_giga_df = sungdong_gu_giga_df.groupby(["BJDONG_NM"])["JIGA"].mean().to_frame().reset_index()
sungdong_gu_dong_giga_df.head()

이제 성동구 동별 평균 공시지가 데이터(`sungdong_gu_dong_giga_df`)를 얻었습니다. 이 데이터는 나중에 시각화를 위해 Folium에서 data로 사용될 예정입니다.

다음은 성동구 정보를 SGIS에서 가져와서 GeoPandas를 이용해 GeoDataFrame을 생성하겠습니다. 생성한 `sungdong_gu_dong_gpd`는 나중에 Folium에서 geo\_data로 사용될 예정입니다.

In [ ]:
accessToken = get_access_token(consumer_key , consumer_secret )
params={
    "accessToken": accessToken,
    "year": 2019,
    "adm_cd": 11040, # 성동구 코드
    "low_search": 1
}

temp_res = requests.get(
    url="https://sgisapi.kostat.go.kr/OpenAPI3/boundary/hadmarea.geojson",
    params=params
)
print('슝~')

In [ ]:
sungdong_gu_dong_gpd = gpd.GeoDataFrame().from_features(temp_res.json())
sungdong_gu_dong_gpd.head()

In [ ]:
# 생성한 dataframe의 정보 확인
sungdong_gu_dong_gpd.info()

공시지가는 법정동 정보를 알아야 매치가 가능한데, 위에서 생성한 `sungdong_gu_dong_gpd`에는 법정동 정보가 없습니다. 참고로 법정동이란 법률로 지정된 명칭과 영역을 지닌 구역으로, 전통적인 지역구분, 주소, 지적(地籍) 분야에서 사용됩니다. 반면 행정동이란 구나 시의 하부 행정구역으로, 우리가 일상에서 사용하는 '동(洞)'을 의미합니다. 예를들어 법정동 코드와 행정동 코드는 법정동과 행정동을 구분하기 위해 숫자로 만든 코드입니다.

`sungdong_gu_dong_gpd` 데이터에 법정동 정보를 추가하기 위해 행정동 정보와 법정동 정보가 맵핑된 `administration_zone_20201001.csv`를 DataFrame로 변환한 후, 행정구역 경계 데이터와 `merge()`를 이용하여 JOIN할 것입니다.

로컬에서 사용하실 분들은 아래 링크에서 데이터를 다운받아주시기 바랍니다.

우선 행정동 정보와 법정동 정보가 맵핑된 csv 파일을 DataFrame으로 가져옵니다.

In [ ]:
path = os.getenv('HOME') + '/aiffel/socar_gis/data/administration_zone_20201001.csv'
address_code_mappint_df = pd.read_csv(path)
address_code_mappint_df.head()

행정구역코드의 데이터 타입을 float에서 object로 변경합니다.

In [ ]:
address_code_mappint_df["행정구역코드"] = address_code_mappint_df["행정구역코드"].fillna(0).astype(int).astype(str)
address_code_mappint_df.head()

필요한 컬럼만 남기고, 컬럼명을 변경해 줍니다.

In [ ]:
address_code_mappint_df = address_code_mappint_df[["시도", "시군구", "행정동(행정기관명)", "법정동", "행정구역코드"]]
address_code_mappint_df.head()

In [ ]:
address_code_mappint_df.rename(
    columns={
        "시도": "si_do",
        "시군구": "si_gun_gu",
        "행정동(행정기관명)": "hj_nm",
        "법정동": "bj_nm",
        "행정구역코드": "adm_cd"
    },
    inplace=True
)

address_code_mappint_df.head()


성동구 행정동&법정동 DataFrame과 행정구역 경계 DataFrame을 `merge()`를 이용해 합친 후, 컬럼의 순서를 보기 좋게 변경해 줍니다.

In [ ]:
sungdong_gu_dong_gpd = pd.merge(sungdong_gu_dong_gpd, address_code_mappint_df, how="left", on="adm_cd")
sungdong_gu_dong_gpd.head()

In [ ]:
sungdong_gu_dong_gpd = sungdong_gu_dong_gpd[["adm_cd", "adm_nm", "si_do", "si_gun_gu", "hj_nm", "bj_nm", "x", "y", "geometry"]]
sungdong_gu_dong_gpd.head()


이제 법정동 이름이 추가된 형태의 데이터인 `sungdong_gu_dong_gpd`가 완성되었습니다. 이제 데이터에 좌표계를 정의하고 좌표계를 UTM-K → WSG84로 변경합니다.


In [ ]:
# 좌표계 정의: UTM-K(epsg:5179)
sungdong_gu_dong_gpd.crs = {"init": "epsg:5179"}

# 좌표계 변환: UTM-K(epsg:5179) -> WSG84(epsg:4326)
sungdong_gu_dong_gpd = sungdong_gu_dong_gpd.to_crs({"init": "epsg:4326"})
sungdong_gu_dong_gpd.head()

GeoDataFrame의 인덱스를 동이름으로 변경합니다.


In [ ]:
sungdong_gu_dong_gpd.set_index(
    "bj_nm",
    drop=False,
    inplace=True
)

sungdong_gu_dong_gpd.head()

In [ ]:
sungdong_gu_dong_gpd[sungdong_gu_dong_gpd["bj_nm"] == "금호동1가"]

### 시각화

___

마지막으로 Folium을 이용하기 위해 기본 맵을 생성합니다. 기본 맵을 생성할 때는 중심부 위도와 경도를 입력해줘야 합니다. 서울시 데이터를 이용하므로 서울시 중심부 위도와 경도인 37.541, 126.986을 입력해 줍니다.

그 후 Folium의 `Choropleth` 메소드를 이용해 앞서 생성한 geo-data인 `sungdong_gu_dong_gpd`와 데이터`sungdong_gu_dong_giga_df`을 합쳐 우리가 원하는 데이터인 서울시 성동구 동별 평균 공시지가를 시각화합니다.

자세한 설명은 아래의 자료를 참고하세요.

-   [Folium documentation](https://python-visualization.github.io/folium/quickstart.html#Choropleth-maps)
-   [\[지도 데이터 시각화\] Part 4. Folium 살펴보기](https://dailyheumsi.tistory.com/m/144?category=854906)

In [ ]:
# 서울시 중심부 위도, 경도
center = [37.541, 126.986]

# map이 서울 중심부에 위치하고, zoom 레벨은 10으로 시작합니다.
map = folium.Map(
    location=center,
    zoom_start=10,
    )
print('슝~')

In [ ]:
folium.Choropleth(
    geo_data=sungdong_gu_dong_gpd,
    name="서울시 성동구 동별 평균 공시지가",
    data=sungdong_gu_dong_giga_df,
    columns=["BJDONG_NM", "JIGA"],
    key_on="feature.id",
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    line_color='white', 
    line_weight=0,
    highlight=False, 
    smooth_factor=1.0,
    legend_name="평균 공시지가",
).add_to(map)

In [ ]:
map

## 4-3. 서울시 대학교 데이터 수집 및 시각화

이번에는 서울 열린데이터광장에서 서울시 대학 및 전문대학 정보를 가져와서 시각화해보겠습니다.

### 데이터 수집 및 전처리

___

-   [서울시 대학 및 전문대학 DB 정보 (한국어)](http://data.seoul.go.kr/dataList/OA-12974/S/1/datasetView.do)

먼저 서울 열린데이터광장에서 가져온 서울시 대학교 데이터를 리스트 형태로 반환하는 함수인 `get_university_list`를 정의하겠습니다. 그 후 서울시 대학정보를 DataFrame으로 생성하겠습니다.

In [ ]:
# 서울시 대학교 데이터 리스트 형태로 가지고 오는 함수
def get_university_list() -> None:
    url="http://openapi.seoul.go.kr:8088/5274694f4f6d6f6e33395743465a61/json/SebcCollegeInfoKor/1/1000/"

    temp_res = requests.get(
        url=url,
        headers={
            "Content-Type": "application/json; chearset=utf-8"
            },
    )

    temp_result_list = temp_res.json()["SebcCollegeInfoKor"]["row"]

    return temp_result_list

print('슝~')

In [ ]:
temp_seoul_university_list = get_university_list()
seoul_university_df = pd.DataFrame(temp_seoul_university_list)
seoul_university_df.head()

In [ ]:
seoul_university_df.info()

DataFrame 컬럼 중 `STATE`는 현재 학교의 운영상태를 알려줍니다. 폐교한 학교의 정보는 필요없으니 `STATE`컬럼이 폐교인 데이터는 제외합니다.

In [ ]:
# 현재 학교 운영상태 확인
set(seoul_university_df["STATE"].tolist())

In [ ]:
# 운영상태가 폐교가 아닌 학교 필터링
seoul_university_df = seoul_university_df[seoul_university_df["STATE"]!="폐교"]
seoul_university_df.head()

혹시 모르니 `STATE`컬럼을 확인해 폐교된 대학교의 데이터가 남아있는지 다시 한 번 확인합니다.

In [ ]:
set(seoul_university_df["STATE"].tolist())

우리가 생성한 DataFrame을 살펴보면 지리정보가 없습니다. 학교주소를 이용해 위도와 경도 데이터를 새로운 컬럼으로 추가합시다.

먼저 주소를 입력하면 주소의 위도와 경도를 반환해주는 함수인 `get_adress_x_y`을 정의합니다.

In [ ]:
# 주소를 입력해 주소의 위도, 경도를 가지고 오는 함수
def get_adress_x_y(address) -> None:
    # SGIS access_token 가져오기
    access_token = get_access_token(consumer_key , consumer_secret)
    url="https://sgisapi.kostat.go.kr/OpenAPI3/addr/geocode.json"
    params={
        "accessToken": access_token,
        "address": address
    }

    temp_res = requests.get(
        url=url,
        params=params
    ).json()["result"]["resultdata"]

    return temp_res[0]["x"], temp_res[0]["y"]

print('슝~')

주소를 조회해 위도와 경도 좌표값을 얻어 컬럼으로 추가합니다.

In [ ]:
for _, row in seoul_university_df.iterrows():
    try:
        x, y = get_adress_x_y(row["ADD_KOR"])
    
        seoul_university_df.loc[_, "x"] = x
        seoul_university_df.loc[_, "y"] = y
    except:
        pass
seoul_university_df.head()



현재 좌표계가 UTM-K이므로 WSG84좌표계로 변환합니다.

In [ ]:
# SGIS 좌표변환 API를 활용한 좌표변환 함수
def get_transcoord(src, dst, posX, posY) -> None:
    access_token = get_access_token(consumer_key, consumer_secret)
    url="https://sgisapi.kostat.go.kr/OpenAPI3/transformation/transcoord.json"
    params={
        "accessToken": access_token,
        "src": src, # 현재 좌표체계
        "dst": dst, # 변환할 좌표체계
        "posX": posX, # 변환할 좌표 X값
        "posY":	posY # 변환할 좌표 Y값
    }

    temp_dict = requests.get(
        url=url,
        params=params
    ).json()["result"]

    return temp_dict["posX"], temp_dict["posY"] # 변환된 좌표값 딕셔너리의 value 리턴

print('슝~')

In [ ]:
for _, row in seoul_university_df.iterrows():
    try:
        x, y = get_transcoord("EPSG:5179", "EPSG:4326", row["x"], row["y"])
    
        seoul_university_df.loc[_, "lng"] = x
        seoul_university_df.loc[_, "lat"] = y

    except:
        pass
seoul_university_df.head()

### 시각화

___

마지막으로 Folium 초기 Map을 생성하고 서울시 대학 정보를 시각화합니다.

In [ ]:
# 초기 Map 메소드에는 초기 중심값 좌표가 들어가야 합니다.

# 서울시 중심부 위도, 경도
center = [37.541, 126.986]

# map이 서울 중심부에 위치하고, zoom 레벨은 12로 시작합니다.
map = folium.Map(
    location=center,
    zoom_start=12

)

print('슝~')

In [ ]:
# 대학 이름, 위도, 경도를 가져와 시각화
for _, row in seoul_university_df.iterrows():
    try:
        folium.Marker(
            location=[row["lat"], row["lng"]], # 좌표
            popup=row["NAME_KOR"], # 클릭시 표시될 popup 내용
            icon=folium.Icon(color="cadetblue", icon="info-sign") # 표시될 아이콘 설정
        ).add_to(map)
    except:
        pass

print('슝~')

In [ ]:
map

## 4-4. 전국주차장 데이터 시각화

다음은 전국주차장 정보 표준데이터를 이용해 시각화를 진행하겠습니다.

### 데이터 전처리

___

로컬에서 사용하기 위해서는 아래에서 `전국주차장 정보 표준데이터`를 다운로드받기 바랍니다. 전국주차장 정보 표준데이터 관련 내용은 아래 링크를 참고해주세요.

-   [전국주차장정보표준데이터](https://www.data.go.kr/data/15012896/standard.do)

우선 전국 주차장 정보 표준 데이터를 불러와 위도와 경도의 데이터 타입을 확인합니다.


In [ ]:
paking_station_info_df = pd.read_csv(os.getenv('HOME') + "/aiffel/socar_gis/data/parking_lot.csv", encoding="cp949")

paking_station_info_df.info()


만약 위도, 경도 데이터 타입이 문자형인 경우 실수형 데이터 타입으로 변경해줍니다.

### 시각화

___

이제 Folium을 이용해서 초기 Map을 생성합니다. 이제까지 했던 것과 마찬가지로 서울시의 위도와 경도 좌표값을 중심값으로 설정합니다.

주차장의 경우 좁은 지역에 밀집되어 있는 경우가 많기 때문에 그룹화해서 표현해야 합니다. 이를 위해 마커들을 그룹화해주는 MarkerCluster 객체를 만든 후, Folium을 이용해 시각화를 진행합니다.

In [ ]:
# MarkerCluster 객체 만들기
marker_cluster = MarkerCluster().add_to(map)

# 주차장 이름, 위도, 경도를 가져와 maker 객체로 추가
for index_num in range(len(paking_station_info_df)):
    try:
        folium.Marker(
            location=[paking_station_info_df.loc[index_num, "위도"], paking_station_info_df.loc[index_num, "경도"]],
            popup=paking_station_info_df.loc[index_num, "소재지지번주소"],
            icon=folium.Icon(color="cadetblue", icon="ok")
        ).add_to(marker_cluster)
    except:
        pass

print('슝~')


시간이 1분이상 소요될 수 있습니다. 코드 실행 후 잠시 기다려주세요.

In [ ]:
map

1분이상 기다려도 map이 나오지 않는 분들은 `cloud jupyter`에서 아래 코드를 실행하고 결과를 확인 해 주세요

In [ ]:
import requests
import json
import os
import math
import time
import pandas as pd
import numpy as np
import geopandas as gpd
import folium
import matplotlib.pyplot as plt

from shapely.geometry import mapping, shape, Point, Polygon, LineString
from folium import plugins
from folium.plugins import MarkerCluster, HeatMap

paking_station_info_df = pd.read_csv(os.getenv('HOME') + "/aiffel/socar_gis/data/parking_lot.csv", encoding="cp949")

paking_station_info_df.info()

# 서울시 중심부 위도, 경도
center = [37.541, 126.986]

# map이 서울 중심부에 위치하고, zoom 레벨은 10으로 시작합니다.
map = folium.Map(
    location=center,
    zoom_start=10,
    )
print('슝~')

# MarkerCluster 객체 만들기
marker_cluster = MarkerCluster().add_to(map)

# 주차장 이름, 위도, 경도를 가져와 maker 객체로 추가
for index_num in range(len(paking_station_info_df)):
    try:
        folium.Marker(
            location=[paking_station_info_df.loc[index_num, "위도"], paking_station_info_df.loc[index_num, "경도"]],
            popup=paking_station_info_df.loc[index_num, "소재지지번주소"],
            icon=folium.Icon(color="cadetblue", icon="ok")
        ).add_to(marker_cluster)
    except:
        pass

print('슝~')

map

## 4-5. 서울시 구별 인구수 데이터 수집 및 시각화

서울시에 거주하는 구별 인구수 데이터를 수집한 후 시각화하겠습니다.

### 데이터 수집 및 전처리

___

먼저 서울시의 구단위 경계 데이터를 불러옵니다.

In [ ]:
seoul_sig_geo_data_path = os.getenv('HOME') + '/aiffel/socar_gis/data/korea_seoul_sig.json'
seoul_sig_geo_data = json.load(
    open(seoul_sig_geo_data_path, encoding="utf-8")
)

print('슝~')

SGIS API를 사용해야하기 때문에 access token을 리턴해주는 함수를 정의해줘야 합니다.

In [ ]:
### SGIS ACCESS TOKEN을 리턴 해주는 함수
def get_access_token(consumer_key, consumer_secret) -> None:
    temp_res = requests.get(
        url="https://sgisapi.kostat.go.kr/OpenAPI3/auth/authentication.json",
        params={
            "consumer_key": consumer_key,
            "consumer_secret": consumer_secret
        }
    )

    access_token = temp_res.json()["result"]["accessToken"]
  
    return access_token

print('슝~')

서울시에 거주하는 인구에 대한 데이터가 필요하기 때문에 서울시 구단위 인구주택총조사 정보를 가져오는 함수인 `get_population_json`을 정의합니다.

In [ ]:
### 서울시 구단위 인구주택총조사 정보 
# 응답정보 컬럼 설명 링크: https://sgis.kostat.go.kr/developer/html/openApi/api/data.html#4
def get_population_json(access_token, si_code) -> None:
    url="https://sgisapi.kostat.go.kr/OpenAPI3/stats/population.json"
    params={
        "accessToken": access_token,
        "year": 2019, # 데이터 기준연도 범위: 2000 ~ 2019 / 2019가 가장 최근 데이터
        "adm_cd": si_code, # 행정구역코드
        "low_search": 1 # 행정구역코드에 해당하는 정보만 요청: 0, 1단계 하위 행정구역 정보 요청: 1, 2단계 하위 행정구역 정보 요청: 2
    }

    temp_population_json = requests.get(
        url=url,
        params=params
    ).json()["result"]

    return temp_population_json

print('슝~')

그 후 DataFrame을 생성하고 필요한 컬럼인 구코드(adm\_cd), 구이름(adm\_nm), 인구수(tot\_ppltn)만 남깁니다. 마지막으로 알아보기 쉽게 하기 위해 컬럼 이름을 아래와 같이 변경합니다.

-   adm\_cd -> gu\_cd
-   adm\_nm -> gu\_nm

In [ ]:
# SGIS access_token 가져오기
access_token = get_access_token(consumer_key, consumer_secret)


# 서울시 구단위 행정구역 인구수 정보 데이터 프레임으로 읽어오기
seoul_gu_pop_df = pd.DataFrame(get_population_json(access_token, 11)) # 서울 시코드: 11


# 서울시 구코드(adm_cd), 구이름(adm_nm), 인구수(tot_ppltn) 컬럼만 남기기
seoul_gu_pop_df = seoul_gu_pop_df[["adm_cd", "adm_nm", "tot_ppltn"]]

# 컬럼명 변경
seoul_gu_pop_df.rename(
    columns={
        "adm_cd": "gu_cd",
        "adm_nm": "gu_nm"
    },
    inplace=True
)

seoul_gu_pop_df.info()

인구수(tot\_ppltn) 컬럼은 데이터 타입을 int로 변경합니다.

In [ ]:
seoul_gu_pop_df["tot_ppltn"] = seoul_gu_pop_df["tot_ppltn"].astype(int)
seoul_gu_pop_df.info()

### 시각화

___

이제 Folium의 초기 Map을 생성하고 서울시 구단위 인구수를 시각화합시다.

In [ ]:
# 서울시 중심부 위도, 경도
center = [37.541, 126.986]

# map이 서울 중심부에 위치하고, zoom 레벨은 10으로 시작 합니다.
map = folium.Map(
    location=center,
    zoom_start=10,
    )

print('슝~')

In [ ]:
folium.Choropleth(
    geo_data=seoul_sig_geo_data,
    name="서울시 구단위 인구수",
    data=seoul_gu_pop_df,
    columns=["gu_nm", "tot_ppltn"],
    key_on="feature.id",
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    line_color='white', 
    line_weight=0,
    highlight=False, 
    smooth_factor=1.0,
    legend_name="인구수",
).add_to(map)

map

## 4-6. 프로젝트(1) 서울시 쏘카존 데이터를 활용한 시각화

지금까지 렉처에서 배운 내용을 기반으로 다양한 공간 데이터 시각화 응용을 진행했습니다. 이번에는 배운 내용을 이용해 다음의 프로젝트를 해결해봅시다.

1.  서울시 쏘카존 데이터를 활용한 시각화
2.  서울시 카페와 버스 정류장 밀도 데이터를 이용한 시각화
3.  2번의 시각화 자료에 쏘카존 데이터 추가해 표현하기

3번 프로젝트에서 추가 데이터로는 따릉이, 쏘카존 마커 등 원작자가 제공하는 데이터를 사용해도 되고, 여러분이 추가로 얻은 데이터를 활용해도 무관합니다.

먼저 주요 라이브러리들의 버전을 살펴보겠습니다.

In [ ]:
import pandas
import geopandas
import folium
import requests
import json
import os
import math
import time
import pandas as pd
import numpy as np
import geopandas as gpd
import folium
import matplotlib.pyplot as plt

from shapely.geometry import mapping, shape, Point, Polygon, LineString
from folium import plugins
from folium.plugins import MarkerCluster, HeatMap
print(pandas.__version__)
print(geopandas.__version__)
print(folium.__version__)

### 공간 데이터를 활용한 시각화 방법

___

공간 데이터를 활용해 시각화하는 방법에는 다양한 내용이 포함되어 있었지만 시각화의 기능적인 순서는 아래와 같이 정리할 수 있습니다.

1.  csv/API 등을 이용해 DataFrame으로 만들기 위한 전처리
2.  DataFrame 생성 후 데이터 전처리
3.  위도 경도 데이터 타입 확인
4.  좌표계 확인(WSG84)
5.  Folium을 이용해 시각화

학습한 내용을 바탕으로 프로젝트를 진행하겠습니다.

## 서울시 쏘카존 데이터를 활용한 시각화

___

로컬에서 진행하기 위해서는 아래 링크를 통해 파일을 다운로드하시기 바랍니다.

먼저 쏘카존 정보가 담긴 CSV 파일을 DataFrame으로 읽어옵니다.

In [ ]:
path = os.getenv('HOME') + '/aiffel/socar_gis/data/20210823_socar_zone.csv'

socar_zone_info_df = pd.read_csv(path)
socar_zone_info_df.head()

Folium 초기 Map 객체를 생성 합니다. 초기 Map 메소드에는 초기 중심값 좌표가 들어가야 합니다. map의 위치는 center에 위치하고, zoom 레벨은 12로 시작합니다.

In [ ]:
# 서울시 중심부 위도, 경도
center = [37.541, 126.986]

map = folium.Map(
    location=center,
    zoom_start=12,
)

print('슝~')

마커들을 그룹화해주는 MarkerCluster 객체를 만들고, 쏘카존 이름, 위도, 경도를 가져와 Marker 객체로 추가해 줍니다.

In [ ]:
# MarkerCluster 객체 만들기
marker_cluster = MarkerCluster().add_to(map)

#maker 객체 추가
for _, row in socar_zone_info_df.iterrows():
    try:
    folium.Marker(
        location=[row["lat"], row["lng"]],# 좌표
        popup = row["zone_name"],# 클릭시 표시될 popup 내용
        icon=folium.Icon(color="cadetblue", icon="car", prefix="fa")
    ).add_to(marker_cluster)
    except:
        pass

print('슝~')

In [ ]:
# 생성한 맵을 확인
map

**시각화 결과 예시**

![content img](https://d3s0tskafalll9.cloudfront.net/media/images/seoul_socar_zone.max-800x600.jpg)

## 4-7. 프로젝트(2) 서울시 카페와 버스 정류장 밀도 데이터를 이용한 시각화

2번째 프로젝트는 서울시 카페 밀도 데이터와 버스 정류장 밀도 데이터를 이용한 시각화입니다. 서울시 상가 데이터는 [소상공인시장진흥공단\_상가(상권)정보](https://www.data.go.kr/data/15083033/fileData.do)에서 다운로드 받으면 됩니다. 로컬에서 사용하기 위해서는 데이터를 다운로드 받아 압축을 해제한 후 `소상공인시장진흥공단_상가(상권)정보_서울_202106.csv` 형식의 파일을 이용하세요.

## 서울시 카페와 버스 정류장 밀도 데이터를 이용한 시각화

___

먼저 서울시 카페 밀도를 시각화하기 위해 상가 정보 데이터를 불러옵니다.

In [ ]:
temp_seoul_store_df = pd.read_csv(os. getenv('HOME') + '/aiffel/socar_gis/data/seoul_store_info_202106.csv'
)

temp_seoul_store_df.head()

우리는 카페 정보를 알고 싶기 때문에 데이터 프레임에서 `상권업종소분류코드` 컬럼에서 'Q12A01'만 필터링합니다. 참고로 Q12A01는 커피점/카페 분류코드입니다. 상가정보업종 코드는 [소상공인시장진흥공단\_상가(상권)정보 업종코드](https://www.data.go.kr/data/15067631/fileData.do) 을 참고하세요.

In [ ]:
seoul_cafe_store_df = temp_seoul_store_df[temp_seoul_store_df["상권업종소분류코드"] == "Q12A01"]

seoul_cafe_store_df.head()

위도와 경도의 데이터 타입을 확인합니다. 데이터 타입이 `float`여야합니다.

In [ ]:
seoul_cafe_store_df.info()

카페 밀도를 표현하기 위해 위치데이터 리스트를 생성합니다.

In [ ]:
seoul_cafe_location_data = [[row["위도"], row["경도"]] for _, row in seoul_cafe_store_df.iterrows()]

print("슝~")

이제 서울시 버스 정류장 밀도 데이터 리스트를 생성합니다.

로컬에서 진행할 경우 아래 링크를 통해 데이터를 다운로드 받기 바랍니다.

서울공공데이터 홈페이지에서 다운로드 받은 버스노선 정보 csv 파일을 DataFrame으로 읽어옵니다.

In [ ]:
path = os.getenv('HOME') + '/aiffel/socar_gis/data/seoul_bus_line_info.csv'
bus_line_info_df = pd.read_csv(path)
bus_line_info_df.head()

컬럼 명을 'X좌표와 Y좌표'에서 '경도와 위도'로 변경하고, 버스 정류장의 밀도를 표현해주는 위치데이터 리스트를 만듭니다.

In [ ]:
# 컬럼명 변경
bus_line_info_df.rename(
    columns={
        "Y좌표": "lat",
        "X좌표": "lng",
    },
    inplace=True
)

bus_line_info_df.info()

In [ ]:
# 버스 정류장 밀도를 표현해주기 위한 위치데이터 리스트 만들기
bus_station_location_data =[[row['lat'], row['lng']] for _, row in bus_line_info_df.iterrows()] ### YOUR CODE ###
print('슝~')


### Heatmap 형태로 시각화하기

___

시각화를 위해 Heatmap을 사용하는 이유는 유동성과 집객효과가 큰 곳을 확인해 쏘카존이 입지하기 유리한 조건을 확인하기 위해서입니다. 상업시설의 대표격인 카페가 많이 모여 있거나 대중교통이 잘 발달된 곳이 유동성과 집객효과가 큰 곳이므로 Heatmap을 통해 카페와 버스 정류장의 밀도를 시각적으로 확인해 보도록 하겠습니다.

Heatmap을 사용하는 방법을 크게 3 단계로 본다면 아래와 같습니다.

1.  Folium 초기 Map 객체를 생성합니다. 초기 Map 메소드에는 초기 중심값 좌표가 들어가야 합니다.
2.  tiles 파라미터를 "Stamen Terrain"로 바꿔줍니다.
3.  zoom\_start를 15로 설정합니다.

In [ ]:
# Folium 초기 Map 객체 생성 

# 서울시 중심부 위도, 경도
center = [37.541, 126.986]

map = folium.Map(
    location=center,
    tiles="Stamen Terrain", ### YOUR CODE ###
    zoom_start=15 ### YOUR CODE ###
)
print('슝~')

이제 Folium의 plugins 메소드를 이용해 다양한 인자를 하나의 Map에 시각화해 봅니다. Folium의 plugins 메소드에 대해 더 알고 싶다면 아래의 자료를 참고하세요.

-   [Folium documentation - plugins](https://python-visualization.github.io/folium/plugins.html)

우선 레이어 적용을 위해 그룹을 만들고, 레이어 콘트롤 박스를 만듭니다.


In [ ]:
# 레이어 적용을 위한 그룹 만들기
# 대표 그룹 만들기
fg = folium.FeatureGroup(name="groups")
map.add_child(fg)

# 카페 그룹 만들기
cafe = plugins.FeatureGroupSubGroup(fg, "cafe_density")
map.add_child(cafe)

# 버스정류장 그룹 만들기
bus = plugins.FeatureGroupSubGroup(fg, 'bus_density')
map.add_child(bus)
### YOUR CODE ###

# 레이어 콘트롤 박스 만들기
folium.LayerControl(collapsed=False).add_to(map)

카페와 버스 정류장의 밀도를 Heatmap에 추가한 후 시각화합니다.

In [ ]:
# 카페 밀도
plugins.HeatMap(seoul_cafe_location_data).add_to(cafe)

In [ ]:
# 버스정류장 밀도
plugins.HeatMap(bus_station_location_data).add_to(bus)
### YOUR CODE ###

In [ ]:
# 생성한 맵 확인
map











**시각화 결과 예시**

![content img](https://d3s0tskafalll9.cloudfront.net/media/images/seoul_cafe_bus_heatmap.max-800x600.jpg)

## 4-8. 프로젝트(3) 시각화 자료에 쏘카존 데이터 추가하기

프로젝트 (1)에서 진행한 쏘카존 데이터를 프로젝트 (2)에서 진행한 시각화 자료에 추가해보겠습니다.

**시각화가 모두 완료된 후에는 쏘카존에 대한 평가 및 선정을 진행합니다.**

## 쏘카존 데이터를 시각화 자료에 추가하기

___

쏘카존 데이터를 프로젝트 (2)의 시각화 자료에 추가하는 방법은 아래와 같습니다.

1.  Folium 사용을 위해 초기 맵 생성(프로젝트 (2) 참고)
2.  Heatmap 형식으로 생성할 레이어 적용을 위한 그룹만들기
3.  쏘카존 마커 클러스터 만들기
4.  카페 그룹, 버스 정류장 그룹, 쏘카존 그룹, 레이어 컨트롤 박스 만들기
5.  카페 밀도 맵에 추가하기
6.  버스 정류장 밀도 맵에 추가하기
7.  쏘카존 마커 맵에 추가하기(프로젝트 (1) 참고)
8.  생성한 맵 확인

### 1\. Folium 사용을 위해 초기 맵 생성(프로젝트 (2) 참고)

In [ ]:
# Folium 초기 Map 객체를 생성 합니다.
# 초기 Map 메소드에는 초기 중심값 좌표가 들어가야 합니다.
map = folium.Map(
    location = center,
    zoom_start = 15)

# 서울시 중심부 위도, 경도
center = [37.541, 126.986]

map = folium.Map(
    location = center,
    zoom_start = 15)
print("슝~")

### 2\. Heatmap 형식으로 생성할 레이어 적용을 위한 그룹만들기

In [ ]:
# 레이어 적용을 위한 그룹 만들기
# 대표 그룹 만들기
fg = folium.FeatureGroup(name='groups')
map.add_child(fg)

### 3\. 쏘카존 마커 클러스터 만들기

In [ ]:
# 쏘카존 마커 클러스터 만들기
mg = plugins.MarkerCluster(
    control=False
    )

map.add_child(mg)


### 4\. 카페 그룹, 버스 정류장 그룹, 쏘카존 그룹, 레이어 컨트롤 박스 만들기

In [ ]:
# 카페 그룹 만들기
cafe = plugins.FeatureGroupSubGroup(fg, 'cafe_density')
map.add_child(cafe)

# 버스 정류장 그룹
bus_station = plugins.FeatureGroupSubGroup(fg, 'bus_station_density')
map.add_child(bus_station)

# 쏘카존 마커 그룹
socar_zone = plugins.FeatureGroupSubGroup(fg, 'socar_zone_density')
map.add_child(socar_zone)

# 레이어 컨트롤 박스
folium.LayerControl(collapsed=False).add_to(map)

### 5\. 카페 밀도 맵에 추가하기

In [ ]:
# 카페 밀도
### YOUR CODE ###
plugins.HeatMap(seoul_cafe_location_data).add_to(cafe)

### 6\. 버스 정류장 밀도 맵에 추가하기

In [ ]:
# 버스정류장 밀도
### YOUR CODE ###
plugins.HeatMap(bus_station_location_data).add_to(bus_station)

### 7\. 쏘카존 마커 맵에 추가하기(프로젝트 (1) 참고)

In [ ]:
# 쏘카존 마커
# MarkerCluster 객체 만들기
marker_cluster = MarkerCluster().add_to(map)

# maker 객체 추가
for _, row in socar_zone_info_df.iterrows():
    try:
        folium.Marker(
            location=[row["lat"], row["lng"]], # 좌표
            popup = row["zone_name"], # 클릭시 표시될 popup 내용
            icon = folium.Icon(color='cadetblue', icon='car', prefix = 'fa')
        ).add_to(marker_cluster)
    except:
        pass

print("슝~")

### 8\. 생성한 맵 확인

In [ ]:
map

**시각화 결과 예시**

![content img](https://d3s0tskafalll9.cloudfront.net/media/images/seoul_cafe_bus_socarzone.max-800x600.jpg)

## 쏘카존에 대한 평가 및 선정

___

완성된 시각화 자료를 토대로 자신만의 분석을 진행해주시길 바랍니다. 만약 여러분이 쏘카존 담당자라고 가정하고 현재 쏘카존의 위치가 적절한 지 판단해봅시다. 여러 장소를 확인해보세요! 신규 쏘카존의 적절한 위치는 어디가 좋을지 고민하여 이미지로 표시하고 그 이유를 적어봅시다.

#### Q. 여러분이 쏘카존 담당이라고 할 때 완성된 시각화 자료를 토대로 현존 쏘카존에 대한 타당성 평가를 진행하고 신규 쏘카존 선정 및 선정 이유를 설명하시오. (github에 업로드할 ipynb파일에 질문과 답변을 작성하세요.)

현재 쏘카존의 위치가 적절한 지 그렇지 않은지 판단해서 서술하고, 신규 쏘카존으로 적절한 곳을 표시해 이미지로 첨부 및 선정 이유를 서술하시오.